In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Custom Sample Dataset
ratings_data = {
    'User_ID': [1, 1, 2, 2, 3, 3, 4, 4, 5, 5,6,6],
    'Movie_ID': [101, 102, 101, 103, 102, 103, 104, 105, 101, 105, 104, 103],
    'Rating': [4.0, 3.5, 4.5, 5.0, 3.0, 3.5, 4.0, 4.5, 3.5, 4.0, 3.0, 4.5]
}
ratings_df = pd.DataFrame(ratings_data)

user_movie_matrix = ratings_df.pivot(index='User_ID', columns='Movie_ID', values='Rating').fillna(0) # fill null values with 0.

user_similarity = cosine_similarity(user_movie_matrix)# cosine similarity between users

# Display the user similarity matrix
print("User Similarity Matrix:")
print(user_similarity)
print(user_movie_matrix)


User Similarity Matrix:
[[1.         0.50344727 0.42854912 0.         0.49557522 0.        ]
 [0.50344727 1.         0.56435099 0.         0.44051636 0.61845811]
 [0.42854912 0.56435099 1.         0.         0.         0.63173968]
 [0.         0.         0.         1.         0.56248283 0.36852275]
 [0.49557522 0.44051636 0.         0.56248283 1.         0.        ]
 [0.         0.61845811 0.63173968 0.36852275 0.         1.        ]]
Movie_ID  101  102  103  104  105
User_ID                          
1         4.0  3.5  0.0  0.0  0.0
2         4.5  0.0  5.0  0.0  0.0
3         0.0  3.0  3.5  0.0  0.0
4         0.0  0.0  0.0  4.0  4.5
5         3.5  0.0  0.0  0.0  4.0
6         0.0  0.0  4.5  3.0  0.0


In [ ]:
def recommend_movies(user_id, user_similarity, user_movie_matrix, top_n=10):
    similar_users = user_similarity[user_id - 1]  # User id.
    similar_users_indices = similar_users.argsort()[::-1][1:]  # Remove the user data, by desc order(cosine similarty value 1).
    #print(similar_users_indices)
    unseen_movies = set(user_movie_matrix.columns[user_movie_matrix.loc[user_id] == 0])

    movie_scores = []
    for movie_id in unseen_movies:
        weighted_rating = 0
        similarity_sum = 0
        for sim_user_idx in similar_users_indices:
            if user_movie_matrix.at[sim_user_idx + 1, movie_id] != 0:
                weighted_rating += user_similarity[user_id - 1, sim_user_idx] * user_movie_matrix.at[sim_user_idx + 1, movie_id]
                similarity_sum += user_similarity[user_id - 1, sim_user_idx]
        if similarity_sum != 0:
            movie_scores.append((movie_id, weighted_rating / similarity_sum))

    movie_scores.sort(key=lambda x: x[1], reverse=True)
    recommended_movies = [movie[0] for movie in movie_scores[:top_n]]

    return recommended_movies




Recommended movies for User 4: [103, 101]


Recommendation for given user:

In [ ]:

user_id = int(input("Please enter the user id (1 to 6)"))
recommended_movies = recommend_movies(user_id, user_similarity, user_movie_matrix)

print(f"Recommended movies for User {user_id}: {recommended_movies}")

Please enter the user id (1 to 6)5
Recommended movies for User 5: [103, 104, 102]


Evaluation:

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score


def evaluate_recommendations(test_user_ids, user_similarity, user_movie_matrix, top_n=5):
    all_predicted = []
    all_actual = []

    for user_id in test_user_ids:
        recommended_movies = recommend_movies(user_id, user_similarity, user_movie_matrix, top_n)
        actual_movies = test_user_movie_matrix.loc[user_id][test_user_movie_matrix.loc[user_id] > 0].index.tolist()

        all_predicted.append(recommended_movies)
        all_actual.append(actual_movies)


    all_predicted_flat = [movie for movies in all_predicted for movie in movies]
    all_actual_flat = [movie for movies in all_actual for movie in movies]


    precision = precision_score(all_actual, all_predicted, average='micro')
    recall = recall_score(all_actual, all_predicted, average='micro')
    f1 = f1_score(all_actual, all_predicted, average='micro')

    return precision, recall, f1


mean_precision, mean_recall, mean_f1 = evaluate_recommendations(test_user_ids, train_user_similarity, train_user_movie_matrix, top_n=5)

print(f"Mean Precision: {mean_precision:.4f}")
print(f"Mean Recall: {mean_recall:.4f}")
print(f"Mean F1-Score: {mean_f1:.4f}")
